In [ ]:
# Import
import pandas as pd
import nimblephysics as nimble
import numpy as np
import os

from COMETH import Skeleton, DynamicSkeleton
from sf_utils import input_sim, rotation, sk_model, dh_model, kalman
from sf_utils.metrics import body_pose_metrics

In [ ]:
#--------File names-------------#
imu_data_path = os.path.abspath('raw_imu_data/imu_s1_acting1.csv')
vicon_data_path = os.path.abspath('./totalcapture/vicon')
subj = 's1'
action = 'acting1'
saving_dir = f'./{subj}_{action}'

#----------Other constants--------#
# Framerate
dt = 1/60

# Camera rotation matrix respect to thorx IMU
theta_cam = np.deg2rad(-75)
R_cam_imu = np.array([[np.cos(theta_cam), 0, np.sin(theta_cam)],[0,1,0],[-np.sin(theta_cam),0,np.cos(theta_cam)]])

# Ratio between camera and imu framerates
ratio = 10

In [ ]:
# Compute ground-truth
target, pos_gt, offset = sk_model.compute_gt(os.path.join(subj, action), vicon_data_path, comp_model=True)

In [ ]:
# Getting accelerometer and gyroscope data from file
imu_cont = input_sim.ImuData(imu_data_path)
imu_cont.read_imu_csv()
imu = imu_cont.acc_read
acc, gyro = imu_cont.get_imu_array(['right_wrist','left_wrist','right_elbow', 'left_elbow', 'sternum'])

# Creating human model for ground-truth data
s = sk_model.create_body_model(os.path.join(subj, action), imu, body_node_list=['ulna_r','ulna_l','humerus_r', 'humerus_l', "thorax","camera"])
s._nimble.setVelocities(np.zeros((49,1)))
s._nimble.setAccelerations(np.zeros((49,1)))
start_pos = s._nimble.getPositions()

# DH parameters calculation
T_list_r, T_list_l = dh_model.compute_list(s)
dh_right, offset_right = dh_model.compute_modified_dh(T_list_r, right=True)
dh_left, offset_left = dh_model.compute_modified_dh(T_list_l, right=False)

# Setting IMU orientation in gt model
T = np.array([np.eye(4), np.eye(4), np.eye(4),np.eye(4), np.eye(4), np.eye(4)])
T[:,:3,:3] = rotation.batch_quat_to_rotmat(imu_cont.get_world_orient(['right_wrist','left_wrist','right_elbow', 'left_elbow', 'sternum','sternum'])[:,0])
sk_model.setImuWorldTransform(s,T)

# Creating ArmModel with parameters 
arm_right = dh_model.ArmModel(dh_right)
arm_left = dh_model.ArmModel(dh_left)

R_r = dh_model.compute_imu_ori(arm_right,s, T, right= True, initial_guess=np.array([0, -np.pi/2, 0, 0, np.pi/2, 0, -np.pi/2, 0]))
R_seg_to_imu_r = [np.eye(3), np.eye(3), R_r[0], np.eye(3), np.eye(3), R_r[1], np.eye(3),R_r[2]] #[R_trunk_IMU, R_upper_IMU, R_fore_IMU]
p_seg_to_imu_r = [np.array([0,0,0]), np.array([0,0,0]), np.array([0.1,0.05,-offset_right[0]+0.1]), np.array([0,0,0]), np.array([0,0,0]), np.array([-offset_right[1]*0.5,0,0]), np.array([0,0,0]), np.array([0,0,-offset_right[2]*0.5])]
R_seg_to_marker_r = np.eye(3) # Assumpt same orient
p_seg_to_marker_r = np.zeros(3)

R_l = dh_model.compute_imu_ori(arm_left, s, T, right= False, initial_guess=np.array([np.pi, -np.pi/2, 0, 0, np.pi/2, 0, -np.pi/2, 0]))
R_seg_to_imu_l = [np.eye(3), np.eye(3), R_l[0], np.eye(3), np.eye(3), R_l[1], np.eye(3), R_l[2]]
p_seg_to_imu_l = [np.array([0,0,0]), np.array([0,0,0]), np.array([0.1,-0.05,-offset_left[0]+0.1]), np.array([0,0,0]), np.array([0,0,0]), np.array([-offset_left[1]*0.5,0,0]), np.array([0,0,0]), np.array([0,0,-offset_left[2]*0.5])] #Riconsidera la prima traslazione sapendo che impatta su dove metti la camera...
R_seg_to_marker_l = np.eye(3) 
p_seg_to_marker_l = np.zeros(3)

R_imu_to_cam = R_cam_imu
p_imu_to_cam = np.zeros(3)

arm_right = dh_model.ArmModel(dh_right, R_seg_to_imu_r, p_seg_to_imu_r,
                        R_seg_to_marker_r, p_seg_to_marker_r,
                        R_imu_to_cam, p_imu_to_cam)
arm_left = dh_model.ArmModel(dh_left, R_seg_to_imu_l, p_seg_to_imu_l,
                    R_seg_to_marker_l, p_seg_to_marker_l,
                    R_imu_to_cam, p_imu_to_cam)


# creation of Q
Q_q = np.eye(8)*1e-5
Q_qd = np.eye(8)*1e-5
Q_qdd = np.eye(8)*1e-5
Q_arm = np.block([[Q_q, np.zeros((8,8)), np.zeros((8,8))], [np.zeros((8,8)), Q_qd, np.zeros((8,8))], [np.zeros((8,8)), np.zeros((8,8)), Q_qdd]] )
Q_whole = np.block([[Q_arm, np.zeros((24, 24))], [np.zeros((24, 24)),Q_arm]])

#Computing initial state throught least square
_, q_right = dh_model.compute_world_transf(arm_right, s, right = True, theta_old=np.array([0, -np.pi/2, 0, 0, np.pi/2, 0, -np.pi/2, 0]))
_, q_left = dh_model.compute_world_transf(arm_left, s, right = False, theta_old= np.array([np.pi, -np.pi/2, 0, 0, np.pi/2, 0, -np.pi/2, 0]))

initial_state = np.zeros(6*8)
initial_state[0:8]=q_right
initial_state[24:32]=q_left

# Kalman filter initialization
ekf = kalman.DualArmEKF([arm_right, arm_left],
                    Q=Q_whole,
                    R_dict={'imu': np.eye(6)*1e-3,
                            'marker_pos': np.eye(3)*1e-6,
                            'marker_rot': np.eye(3)*1e-3,
                            'qr_rot': np.eye(3)*1e-3},
                    initial_state=initial_state)

In [ ]:
pos_estimation = []

for i in range(0, imu['n_frames']):
    
    # Setting gt position
    s._nimble.setPositions(pos_gt[i])
    
    # Creating IMUs measures tuples list
    z_all = [
        (0, 2, np.array([gyro[4,i],acc[4,i]*9.80665]).reshape(6)),   # right arm, thorax
        (0, 5, np.array([gyro[2,i],acc[2,i]*9.80665]).reshape(6)),   # right arm upper-arm 
        (0, 7, np.array([gyro[0,i],acc[0,i]*9.80665]).reshape(6)),   # right arm, forearm 
        (1, 2, np.array([gyro[4,i],acc[4,i]*9.80665]).reshape(6)),   # left_arm, thorax
        (1, 5, np.array([gyro[3,i],acc[3,i]*9.80665]).reshape(6)),   # left_arm, upper-arm
        (1, 7, np.array([gyro[1,i],acc[1,i]*9.80665]).reshape(6)),   # left_arm, forearm
    ]
    
    # Prediction step of kalman filter
    ekf.predict(dt)
    
    # If is a camera frame, do camera stuff
    if i%ratio == 0:
        # Get gt camera orientation and position
        R_imu_w = rotation.batch_quat_to_rotmat(imu_cont.get_world_orient(['sternum'])[:,i]).reshape(3,3)
        R_cam_w = R_imu_w@R_imu_to_cam

        camera_pos = sk_model.getImuWorldPosition(s)[5] + R_imu_w@p_imu_to_cam

        # Get gt wrists orient
        R_l_w = rotation.batch_quat_to_rotmat(imu_cont.get_world_orient(['left_wrist'])[:,i]).reshape(3,3)@R_seg_to_marker_l #(marker -> world) 
        R_r_w = rotation.batch_quat_to_rotmat(imu_cont.get_world_orient(['right_wrist'])[:,i]).reshape(3,3)@R_seg_to_marker_r

        # GT wrists positions and if are visible
        pos_target = (s._nimble.getJointWorldPositions(s.joints)).reshape(-1,3)[[1,7],:] - s._nimble.getBodyNode("thorax").getWorldTransform().matrix()[:3,3] #first is left, second is  right
        pos_l_w, pos_r_w = input_sim.wrist_camera_simulation(pos_target, camera_pos=camera_pos, R_cam=R_cam_w)
        

        # World camera orient gt
        qr_pos = (camera_pos - s._nimble.getBodyNode("thorax").getWorldTransform().matrix()[:3,3])@R_cam_w
        qr_rot = R_cam_w.T

        # If specific data available, do the update with that data
        if not np.any(np.isnan(pos_r_w)):
            try:
                ekf.iekf_update_marker(arm_id=0, z_pos=pos_r_w, z_rot=R_r_w)
            except Exception as e:
                print(e)
        if not np.any(np.isnan(pos_l_w)):
            try:
                ekf.iekf_update_marker(arm_id=1, z_pos=pos_l_w, z_rot=R_l_w)
            except Exception as e:
                print(e)
        try:
            ekf.update_qr(arm_id=0, z_rot=qr_rot)
        except Exception as e:
                print(e)

        try:
            ekf.update_qr(arm_id=1, z_rot=qr_rot)
        except Exception as e:
                print(e)
        
    # Updating with IMUs data
    try:
        ekf.update_imu(z_all, use_joseph=True)
    except Exception as e:
                print(e)

    # Store estimate position
    q_est = (ekf.state.reshape(6,8)[[0,3],:])

    pos_est_right, _, _ = arm_right.forward_kinematics(q_est[0])
    pos_est_left, _, _ = arm_left.forward_kinematics(q_est[1])
    pos_est = np.array(pos_est_right + pos_est_left).reshape(-1,3)[[2,5,7,10,13,15],:]

    pos_estimation.append(pos_est)

In [ ]:
body_pose_metrics.save_metrics_comparative(seq_dir=saving_dir, pos_estimation = pos_estimation, marker = target, offset=offset)